In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 필수 라이브러리 설치
!pip install roboflow singleton-decorator

import os
import sys
import cv2
import numpy as np
import torch
from sklearn.metrics import accuracy_score, f1_score
from singleton_decorator import singleton
import zipfile

# YOLOv5 레포지토리 클론 및 이동
if not os.path.exists('/content/yolov5'):
    !git clone https://github.com/ultralytics/yolov5 /content/yolov5
%cd /content/yolov5
!pip install -r requirements.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/yolov5


In [9]:
from roboflow import Roboflow
import yaml

# Roboflow API 설정 및 데이터셋 다운로드
rf = Roboflow(api_key="gUNvjxIQ9rKYruOxpb3t")

# 프로젝트와 버전 설정
projects = [
    {"workspace": "fpccloud1", "project": "logistics-sz9jr", "version": 2},
    {"workspace": "ludovico-bozzolo-f4hui", "project": "logistics-qikly", "version": 3},
    {"workspace": "fpccloud1", "project": "vehicle-w9bm8", "version": 3},
]

# 모델 파일 경로
model_files = [
    '/content/drive/MyDrive/Colab_Notebooks/SmartMaritimeLogistics/model/best_0.3.0.pt',
    '/content/drive/MyDrive/Colab_Notebooks/SmartMaritimeLogistics/model/best_0.3.1.pt',
    '/content/drive/MyDrive/Colab_Notebooks/SmartMaritimeLogistics/model/best_0.3.2.pt'
]

# 추적 대상 클래스
track_classes = ['car', 'truck', 'van', 'forklift', 'fire', 'smoke']

# 결과 저장 변수
model_results = {}

def load_model(model_path):
    return torch.hub.load('/content/yolov5', 'custom', path=model_path, source='local')

def parse_label_file(label_file, class_names):
    labels = []
    with open(label_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            cls_id = int(parts[0])
            if cls_id in class_names:
                cls_name = class_names[cls_id]
                labels.append(cls_name)
            else:
                print(f"Warning: Class ID {cls_id} not found in class names. Skipping...")
    return labels

def evaluate_model(model, validation_images, validation_labels, conf_threshold=0.25):
    y_true = []
    y_pred = []

    for img_path in validation_images:
        img = cv2.imread(img_path)
        if img is None:
            continue

        # 모델 추론
        results = model(img)

        img_name = os.path.splitext(os.path.basename(img_path))[0]
        label_file = os.path.join(validation_labels, f"{img_name}.txt")

        if not os.path.exists(label_file):
            print(f"Label file {label_file} does not exist. Skipping...")
            continue

        true_labels = parse_label_file(label_file, model.names)
        y_true.extend(true_labels)

        pred_labels = []
        for *xyxy, conf, cls in results.xyxy[0]:
            if conf > conf_threshold:
                class_name = model.names[int(cls)]
                if class_name in track_classes:
                    pred_labels.append(class_name)
        y_pred.extend(pred_labels)

    return y_true, y_pred

def download_dataset(workspace, project, version):
    project = rf.workspace(workspace).project(project)
    version = project.version(version)
    dataset = version.download("yolov5")
    return dataset.location

# 각 프로젝트에 대해 데이터셋 다운로드 및 모델 평가
for project_info in projects:
    workspace = project_info["workspace"]
    project = project_info["project"]
    version = project_info["version"]

    print(f"Downloading dataset for workspace: {workspace}, project: {project}, version: {version}")
    dataset_path = download_dataset(workspace, project, version)

    # 검증 데이터셋 경로
    validation_images_dir = os.path.join(dataset_path, 'valid', 'images')
    validation_labels_dir = os.path.join(dataset_path, 'valid', 'labels')

    validation_images = [os.path.join(validation_images_dir, f) for f in os.listdir(validation_images_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

    for model_file in model_files:
        print(f"Evaluating model: {model_file} on dataset {project}")
        model = load_model(model_file)
        y_true, y_pred = evaluate_model(model, validation_images, validation_labels_dir)

        if not y_true or not y_pred:
            print(f"No valid detections for model {model_file} on dataset {project}.")
            continue

        # y_true와 y_pred의 길이를 일치시키기 위한 보정
        min_length = min(len(y_true), len(y_pred))
        y_true = y_true[:min_length]
        y_pred = y_pred[:min_length]

        accuracy = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average='weighted')

        if model_file not in model_results:
            model_results[model_file] = []

        model_results[model_file].append({
            'project': project,
            'accuracy': accuracy,
            'f1_score': f1
        })

# 결과 출력
for model_file, results in model_results.items():
    print(f"Model: {model_file}")
    for result in results:
        print(f"Dataset: {result['project']}")
        print(f"Accuracy: {result['accuracy']:.4f}")
        print(f"F1 Score: {result['f1_score']:.4f}\n")

loading Roboflow workspace...
loading Roboflow project...


YOLOv5 🚀 v7.0-350-g6096750f Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)

Fusing layers... 


Evaluating model: /content/drive/MyDrive/Colab_Notebooks/SmartMaritimeLogistics/model/best_0.3.0.pt on dataset logistics-sz9jr


Model summary: 157 layers, 7064065 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 
YOLOv5 🚀 v7.0-350-g6096750f Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)



Evaluating model: /content/drive/MyDrive/Colab_Notebooks/SmartMaritimeLogistics/model/best_0.3.1.pt on dataset logistics-sz9jr


Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.


KeyboardInterrupt: 